In [24]:
data = sc.textFile('../data/msnbc/data.csv') \
         .map(lambda line: list(map(int,line.split(',')))) \
         .zipWithIndex()

In [25]:
result = data.collect()

In [26]:
result[:5]

[([1, 1], 0), ([2], 1), ([3, 2, 2, 4, 2, 2, 2, 3, 3], 2), ([5], 3), ([1], 4)]

In [38]:
def function1(x):
    key = x[0]
    seq_id, seq = x[1]
    indices = list()
    for idx in range(len(seq)):
        if seq[idx] == key:
            indices.append(idx)
    return key, (seq_id, seq), indices

In [39]:
def get_app(x):
    key = x[0]
    idx, seq = x[1]
    indices = x[2]
    return key, ((idx, seq), [indices[0], indices[-1]])

In [40]:
ceva = data.map(lambda x: ((x[1],x[0]), set(x[0]))) \
        .flatMapValues(lambda x:x) \
        .map(lambda x: (x[1], x[0])) \
        .map(lambda x: function1(x)) \
        .map(lambda x: get_app(x)) \
        .groupByKey() \
        .mapValues(list)

In [41]:
result2 = ceva.collect()

In [42]:
result2[0]

(2,
 [((1, [2]), [0, 0]),
  ((2, [3, 2, 2, 4, 2, 2, 2, 3, 3]), [1, 6]),
  ((15, [2]), [0, 0]),
  ((24, [2, 6]), [0, 0]),
  ((31, [2, 4, 4, 4, 3]), [0, 0]),
  ((60, [2]), [0, 0]),
  ((63, [2, 12, 3, 4, 12, 12]), [0, 0]),
  ((70, [2]), [0, 0]),
  ((74, [1, 2, 1, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1, 2, 2]),
   [1, 17]),
  ((77, [1, 4, 7, 1, 10, 10, 1, 2, 2, 1, 1, 6, 1]), [7, 8]),
  ((80, [1, 1, 12, 2, 2, 4]), [3, 4]),
  ((106, [4, 4, 10, 10, 10, 10, 2, 2, 2, 2, 10, 12, 12, 10, 7, 7, 1, 10]),
   [6, 9]),
  ((116, [1, 1, 1, 12, 1, 10, 10, 14, 2, 14, 14, 14, 14, 14]), [8, 8]),
  ((136, [6, 9, 9, 2]), [3, 3]),
  ((140, [1, 1, 2]), [2, 2]),
  ((147,
    [11,
     1,
     1,
     1,
     2,
     1,
     1,
     5,
     5,
     5,
     5,
     5,
     5,
     1,
     1,
     6,
     6,
     1,
     1,
     6,
     1,
     1,
     1,
     1,
     5]),
   [4, 4]),
  ((153, [1, 2, 2, 2, 2, 2, 6, 6, 7, 7, 7, 7]), [1, 5]),
  ((157,
    [6,
     2,
     2,
     2,
     15,
     2,
     2